In [120]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
import math


In [121]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, normal_fun=None):
        if (normal_fun):
            return np.c_[np.ones(self.get_n()), normal_fun(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [122]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def get_prediction_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = get_prediction_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)


In [123]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split

    returns (i, train, test)
    """
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results

# Questão 1

Considere o conjunto de dados disponível em **kc2.csv**, organizado em 22 colunas, sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21 atributos são referentes à caracterização de códigos-fontes para processamento de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de defeitos. Maiores detalhes sobre os dados p o dem ser conferidos em https://www.openml.org/d/1063.

In [124]:
data = np.genfromtxt('kc2.csv', delimiter=',')
np.random.seed(666)
folds = k_fold_split(data, 10)
features = np.arange(21)
output = 21

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo:

- **KNN** (escolha k = 1 e k = 5, distância Euclidiana (e Mahalonobis, para a pós-graduação));
- **Árvore de decisão** (você pode usar uma implementação já existente com índices de impureza de gini e entropia)

In [125]:
def euclidean_distance(p, q):
    return np.sqrt(np.sum((p - q)**2))

def mahalanobis_distance(p, q, V):
    """V: matriz de covariância dos dados de treino"""
    return np.sqrt((p - q).T.dot(np.linalg.inv(V)).dot((p - q)))

def kNN(X, y, x_test, y_test, k, distance_func = euclidean_distance):
    y_pred = []

    # encontrar k padrões mais próximos
    for xi in x_test:
        distances = []
        for j in range(X.shape[0]):
            # distances.append(euclidean_distance(X[j, :], xi))
            V = np.cov(X)
            distances.append(mahalanobis_distance(X[j, :], xi, V))
        distances = np.array(distances)

        dist = np.argsort(distances)[:k] 
        labels = y[dist]

        lab = stats.mode(labels) 
        lab = lab.mode[0]
        y_pred.append(lab)

    # computar predição com base na ponderação das saidos dos padrões mais próximos
    return (y_test, np.array(y_pred))

n_folds = 10
accuracy_, precision_, recall_, f1_score_ = np.zeros(n_folds), np.zeros(n_folds), np.zeros(n_folds), np.zeros(n_folds)
for i, train, test in k_fold_train_test(folds):
    train = Set(train, features, output)
    test = Set(test, features, output)
    y_test, y_pred = kNN(train.get_X(normal_fun=stats.zscore), train.get_y(), test.get_X(normal_fun=stats.zscore), test.get_y(), 3)

    accuracy_[i] = (accuracy(y_test, y_pred))
    precision_[i] = (precision(y_test, y_pred))
    recall_[i] = (recall(y_test, y_pred))
    f1_score_[i] = (f1_score(y_test, y_pred))

print("%i-fold cross validation com %s" % (n_folds, 'kNN'))
print("acurácia: %.8f +/- %.8f" % (accuracy_.mean(), accuracy_.std()))
print("revocação: %.8f +/- %.8f" % (precision_.mean(), precision_.std()))
print("precisão: %.8f +/- %.8f" % (recall_.mean(), recall_.std()))
print("f1-score: %.8f +/- %.8f" % (f1_score_.mean(), f1_score_.std()))
print("")

ValueError: shapes (22,) and (469,469) not aligned: 22 (dim 0) != 469 (dim 0)

b) Para cada modelo criado, reporte **valor médio** e **desvio padrão** das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**.